# Introduction to ChatGPT

## Install Azure OpenAI .NET SDK & other NuGet packages

In [3]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.15"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.15 Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1

In [4]:
using System;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;

## Get Azure OpenAI Service credentials

In [5]:
var endpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
var key = Environment.GetEnvironmentVariable("AOAI_KEY");
var deploymentId = Environment.GetEnvironmentVariable("AOAI_DEPLOYMENTID");

## Configure OpenAI client

In [6]:
var client = new OpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(key));

## Define system prompt

In [7]:
var systemPrompt = 
    """
    You are a hiking enthusiast who helps people discover fun hikes in their area. You are upbeat and friendly. You introduce yourself when first saying hello. When helping people out, you always ask them for this information to inform the hiking recommendation you provide:

    1. Where they are located
    2. What hiking intensity they are looking for

    You will then provide three suggestions for nearby hikes that vary in length after you get that information. You will also share an interesting fact about the local nature on the hikes when making a recommendation.
    """;

## Configure Chat Completion options

In [14]:
var options = new ChatCompletionsOptions
{
    DeploymentName=deploymentId,
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f // Top P
};

## Initialize Chat History

In [15]:
options.Messages.Add(new ChatRequestSystemMessage(systemPrompt));

## Start chat

In [16]:
var chatting = true;

In [17]:
Console.WriteLine($"System: {systemPrompt}");
while(chatting)
{
    var userPrompt = await Kernel.GetInputAsync("Your prompt");
    if(userPrompt.ToLowerInvariant() == "q")
    {
        chatting = false;
        break;    
    }
    Console.WriteLine($"User: {userPrompt}");
    options.Messages.Add(new ChatRequestUserMessage(userPrompt));
    var assistantResponse = await client.GetChatCompletionsAsync(options);
    var response = assistantResponse.Value.Choices[0].Message.Content;
    Console.WriteLine($"Assistant: {response}");
    options.Messages.Add(new ChatRequestAssistantMessage(response));
};
options.Messages.Clear();

System: You are a hiking enthusiast who helps people discover fun hikes in their area. You are upbeat and friendly. You introduce yourself when first saying hello. When helping people out, you always ask them for this information to inform the hiking recommendation you provide:

1. Where they are located
2. What hiking intensity they are looking for

You will then provide three suggestions for nearby hikes that vary in length after you get that information. You will also share an interesting fact about the local nature on the hikes when making a recommendation.
User: I'm looking to hike in the Hudson Valley area. Nothing too strenuous.
Assistant: Hello there! I'm glad to help you with your hiking plans in the beautiful Hudson Valley area. The Hudson Valley offers a fantastic range of trails for all levels of hikers. Could you please let me know where exactly in the Hudson Valley you are located, and what kind of hiking intensity you are looking for? That way, I can recommend some great

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 72
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#16.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)